# Data Preprocessing for Simulation

This file was used for creating the datasets used for the car-sharing charging simulation software.

In [2]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import datetime
import json
import psycopg2
import math
import numpy as np
import bisect
from datetime import datetime
import pickle

# load the credentials from the JSON file
with open('config/credentials.json') as f:
    credentials = json.load(f)

connection_string = f"postgresql://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database_name']}"

# create the engine with the connection string
engine = create_engine(connection_string)


# Preprocessing of Reservation Table #

In [25]:
### pre-processing of reservation table: ###
# reservationfrom_discrete: discretize reservationfrom to 15 minutes 
# reservationto_discrete: discretize reservationto to 15 minutes 
# syscreatedate_discrete: discretize syscreatedate to 15 minutes 
# time_booking_to_bookingstart: time difference between syscreatedate and reservationfrom, in 15 min units
# reservation_duration: time difference between reservationfrom_discrete and reservationto_discrete, in 15 min units
# syscreatedate_daytime: time of the day of syscreatedate, in 15 min units
# reservationfrom_daytime: time of the day of reservationfrom, in 15 min units
# reservationto_daytime: time of the day of reservationto, in 15 min units
# booking_trip_same_day: boolean wether a trip was reserved on the same day as the trip or not

# customer bookings
sql = """WITH reservations AS (
SELECT *,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(syscreatedate, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as syscreatedate_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationfrom, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationfrom_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_firststart_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_lastend_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationto, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationto_discrete
FROM mobility.reservation 
)
SELECT *, (drive_km/range)*100 AS required_soc,
EXTRACT(epoch FROM (reservationfrom_discrete - syscreatedate_discrete)) / 900 AS time_booking_to_bookingstart,
EXTRACT(epoch FROM (reservationto_discrete - reservationfrom_discrete)) / 900 AS reservation_duration
FROM reservations JOIN msc_2023_dominik.vehicle_information USING(vehicle_no) 
ORDER BY reservationfrom 
"""

# load table
data = pd.read_sql(sql, engine)

# discretized time of the day in 15 minutes steps (0-95)
data["syscreatedate_daytime"] = data.syscreatedate_discrete.apply(lambda x: x.hour) * 4 + data.syscreatedate_discrete.apply(lambda x: x.minute) / 15
data["reservationfrom_daytime"] = data.reservationfrom_discrete.apply(lambda x: x.hour) * 4 + data.reservationfrom_discrete.apply(lambda x: x.minute) / 15
data["reservationto_daytime"] = data.reservationto_discrete.apply(lambda x: x.hour) * 4 + data.reservationto_discrete.apply(lambda x: x.minute) / 15
data["drive_firststart_daytime"] = data.drive_firststart_discrete.apply(lambda x: x.hour) * 4 + data.drive_firststart_discrete.apply(lambda x: x.minute) / 15
data["drive_lastend_daytime"] = data.drive_lastend_discrete.apply(lambda x: x.hour) * 4 + data.drive_lastend_discrete.apply(lambda x: x.minute) / 15

# add columns with date
data["syscreatedate_discrete_date"] = pd.to_datetime(data['syscreatedate_discrete'])
data["reservationfrom_discrete_date"] = pd.to_datetime(data['reservationfrom'])
data["reservationto_discrete_date"] = pd.to_datetime(data['reservationto'])
data["drive_firststart_discrete_date"] = pd.to_datetime(data['drive_firststart'])
data["drive_lastend_discrete_date"] = pd.to_datetime(data['drive_lastend'])

# clean data, if reservation start is before system creation time
cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime"] = data["reservationfrom_daytime"]
cond = data["reservationfrom_discrete_date"] < data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_discrete_date"] = data["reservationfrom_discrete_date"]

# caclulate number of days since first day of observation period
start_date = pd.to_datetime('2019-01-01')
data['days_since_start_syscreatedate'] = (data['syscreatedate_discrete_date'] - start_date).dt.days
data['days_since_start_reservationfrom'] = (data['reservationfrom_discrete_date'] - start_date).dt.days
data['days_since_start_reservationto'] = (data['reservationto_discrete_date'] - start_date).dt.days
data['days_since_start_drive_firststart'] = (data['drive_firststart_discrete_date'] - start_date).dt.days
data['days_since_start_drive_lastend'] = (data['drive_lastend_discrete_date'] - start_date).dt.days

# assing time ID 0 for all reservations made before 2019, since those time periods are not part of the simulation
cond = data["days_since_start_syscreatedate"] < 0
data.loc[cond, "days_since_start_syscreatedate"] = 0
cond = data["days_since_start_reservationfrom"] < 0
data.loc[cond, "days_since_start_reservationfrom"] = 0
cond = data["days_since_start_reservationto"] < 0
data.loc[cond, "days_since_start_reservationto"] = 0
cond = data["days_since_start_drive_firststart"] < 0
data.loc[cond, "days_since_start_drive_firststart"] = 0
cond = data["days_since_start_drive_lastend"] < 0
data.loc[cond, "days_since_start_drive_lastend"] = 0

# clean data, if reservation start is before system creation time
cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime"] = data["reservationfrom_daytime"]

# clean data, if reservation duration is negative
cond = data["reservation_duration"] < 0
data.loc[cond, "reservation_duration"] = 0

# calculate time ID representations 
data['syscreatedate_time_discrete'] = data['syscreatedate_daytime'] + 96 * data['days_since_start_syscreatedate']
data['reservationfrom_time_discrete'] = data['reservationfrom_daytime'] + 96 * data['days_since_start_reservationfrom']
data['reservationto_time_discrete'] = data['reservationto_daytime'] + 96 * data['days_since_start_reservationto']
data['drive_firststart_time_discrete'] = data['drive_firststart_daytime'] + 96 * data['days_since_start_drive_firststart']
data['drive_lastend_time_discrete'] = data['drive_lastend_daytime'] + 96 * data['days_since_start_drive_lastend']

# set negative required SoC to 0 (calculated from trips with negative driven distance)
cond = data["required_soc"] < 0
data.loc[cond, "required_soc"] = 0

# set trips that require more than 100% SoC to 100%. More charging is not possible
cond = data["required_soc"] > 100
data.loc[cond, "required_soc"] = 100

# set system creation time to 0 if done before 2019
cond = data["syscreatedate_discrete_date"] < pd.to_datetime('2019-01-01')
data.loc[cond, "syscreatedate_time_discrete"] = 0

# save table in database
data.to_sql("reservations_no_service_long_time", engine, schema="msc_2023_dominik", if_exists='fail')

reservations_no_service = data


# Service reservations
sql = """WITH reservations AS (
SELECT *,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(syscreatedate, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as syscreatedate_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationfrom, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationfrom_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_firststart_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_lastend_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationto, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationto_discrete
FROM mobility.service_reservation 
)
SELECT *, (drive_km/range)*100 AS required_soc,
EXTRACT(epoch FROM (reservationfrom_discrete - syscreatedate_discrete)) / 900 AS time_booking_to_bookingstart,
EXTRACT(epoch FROM (reservationto_discrete - reservationfrom_discrete)) / 900 AS reservation_duration
FROM reservations JOIN msc_2023_dominik.vehicle_information USING(vehicle_no) 
ORDER BY reservationfrom 
"""

# load table
data = pd.read_sql(sql, engine)

# discretized time of the day in 15 minutes steps (0-95)
data["syscreatedate_daytime"] = data.syscreatedate_discrete.apply(lambda x: x.hour) * 4 + data.syscreatedate_discrete.apply(lambda x: x.minute) / 15
data["reservationfrom_daytime"] = data.reservationfrom_discrete.apply(lambda x: x.hour) * 4 + data.reservationfrom_discrete.apply(lambda x: x.minute) / 15
data["reservationto_daytime"] = data.reservationto_discrete.apply(lambda x: x.hour) * 4 + data.reservationto_discrete.apply(lambda x: x.minute) / 15
data["drive_firststart_daytime"] = data.drive_firststart_discrete.apply(lambda x: x.hour) * 4 + data.drive_firststart_discrete.apply(lambda x: x.minute) / 15
data["drive_lastend_daytime"] = data.drive_lastend_discrete.apply(lambda x: x.hour) * 4 + data.drive_lastend_discrete.apply(lambda x: x.minute) / 15

# add columns with date
data["syscreatedate_discrete_date"] = pd.to_datetime(data['syscreatedate_discrete'])
data["reservationfrom_discrete_date"] = pd.to_datetime(data['reservationfrom'])
data["reservationto_discrete_date"] = pd.to_datetime(data['reservationto'])
data["drive_firststart_discrete_date"] = pd.to_datetime(data['drive_firststart'])
data["drive_lastend_discrete_date"] = pd.to_datetime(data['drive_lastend'])

# clean data, if reservation start is before system creation time
cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime"] = data["reservationfrom_daytime"]
cond = data["reservationfrom_discrete_date"] < data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_discrete_date"] = data["reservationfrom_discrete_date"]

# caclulate number of days since first day of observation period
start_date = pd.to_datetime('2019-01-01')
data['days_since_start_syscreatedate'] = (data['syscreatedate_discrete_date'] - start_date).dt.days
data['days_since_start_reservationfrom'] = (data['reservationfrom_discrete_date'] - start_date).dt.days
data['days_since_start_reservationto'] = (data['reservationto_discrete_date'] - start_date).dt.days
data['days_since_start_drive_firststart'] = (data['drive_firststart_discrete_date'] - start_date).dt.days
data['days_since_start_drive_lastend'] = (data['drive_lastend_discrete_date'] - start_date).dt.days

# assing time ID 0 for all reservations made before 2019, since those time periods are not part of the simulation
cond = data["days_since_start_syscreatedate"] < 0
data.loc[cond, "days_since_start_syscreatedate"] = 0
cond = data["days_since_start_reservationfrom"] < 0
data.loc[cond, "days_since_start_reservationfrom"] = 0
cond = data["days_since_start_reservationto"] < 0
data.loc[cond, "days_since_start_reservationto"] = 0
cond = data["days_since_start_drive_firststart"] < 0
data.loc[cond, "days_since_start_drive_firststart"] = 0
cond = data["days_since_start_drive_lastend"] < 0
data.loc[cond, "days_since_start_drive_lastend"] = 0

# clean data, if reservation start is before system creation time
cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime"] = data["reservationfrom_daytime"]

# clean data, if reservation duration is negative
cond = data["reservation_duration"] < 0
data.loc[cond, "reservation_duration"] = 0

# set system creation time to 0 if done before 2019
cond = data["syscreatedate_discrete_date"] < pd.to_datetime('2019-01-01')
data.loc[cond, "syscreatedate_time_discrete"] = 0

# calculate time ID representations 
data['syscreatedate_time_discrete'] = data['syscreatedate_daytime'] + 96 * data['days_since_start_syscreatedate']
data['reservationfrom_time_discrete'] = data['reservationfrom_daytime'] + 96 * data['days_since_start_reservationfrom']
data['reservationto_time_discrete'] = data['reservationto_daytime'] + 96 * data['days_since_start_reservationto']
data['drive_firststart_time_discrete'] = data['drive_firststart_daytime'] + 96 * data['days_since_start_drive_firststart']
data['drive_lastend_time_discrete'] = data['drive_lastend_daytime'] + 96 * data['days_since_start_drive_lastend']

# no energy consumption during service trips
data["required_soc"] = 0

# save table in database
data.to_sql("service_reservations_long_time", engine, schema="msc_2023_dominik", if_exists='fail')

# join both tables
reservations_service = data
union_df = pd.concat([reservations_no_service, reservations_service], axis=0)

# save final table in database
union_df.to_sql("reservations_long_time", engine, schema="msc_2023_dominik", if_exists='fail')


884

In [40]:
### create indices for fast data loading in environment ###
conn = psycopg2.connect(dbname=credentials['database_name'], user=credentials['username'], password=credentials['password'], host=credentials['host'])
cur = conn.cursor()
#cur.execute("CREATE INDEX reservationfrom_discrete_date ON msc_2023_dominik.reservations_discrete USING hash(reservationfrom_discrete_date)")
#cur.execute("CREATE INDEX reservationfrom_discrete_index_b_tree ON msc_2023_dominik.reservations_discrete (reservationfrom_discrete)")
#for i in range(0,83):
#    cur.execute("CREATE INDEX vehicle_no_b_tree_index_week_{} ON msc_2023_dominik.discrete_weeks_{} (vehicle_no)".format(i,i))
conn.commit()
cur.close()
conn.close()

# Preprocessing of Station Table #

In [8]:
# Contains all stations in the mobility.station table with valid geometry, that were used during the 
# observation period for bookings (mobility.reservation and mobility.service_reservation)
sql = "WITH stations AS (SELECT DISTINCT start_station_no FROM (SELECT DISTINCT start_station_no FROM mobility.reservation UNION SELECT DISTINCT start_station_no FROM mobility.service_reservation) AS station_union) SELECT station_no, ST_Transform(ST_SetSRID(geom,4326), 2056) AS geom FROM stations LEFT OUTER JOIN mobility.station ON start_station_no = station_no WHERE geom is not NULL"

# get processed data
gdf = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")

# save in database
gdf.to_postgis("distinct_stations", engine, schema="msc_2023_dominik", if_exists='fail')  

# Preprocessing of Vehicle Information Table #

In [3]:
# get information about each car used in the discrete tables
sql = """SELECT vehicle_no, mobility.simulated_ev.model_name, mobility.simulated_ev.brand_name, charge_power, battery_capacity, range, vehicle_category 
         FROM mobility.simulated_ev 
         FULL OUTER JOIN mobility.vehicle USING (vehicle_no) 
         RIGHT OUTER JOIN discrete.discrete_weeks_0 using (vehicle_no)"""

# get processed data
data = pd.read_sql(sql, engine)

# replace some car types, which are not used anymore in a fully electric vehicle fleet 
data = data.replace(["Budget Electro"], 'Budget')
data = data.replace(['Combi Electro'], 'Combi')

# save table in database
data.to_sql("vehicles_preprocessing", engine, schema="msc_2023_dominik", if_exists='fail')


### vehicle types: ###
# get car information for each vehcile category
sql = """WITH vehicles AS (
                            SELECT *
                            FROM msc_2023_dominik.vehicles_preprocessing 
        )
        SELECT model_name, brand_name, charge_power, battery_capacity, range, vehicle_category 
        FROM vehicles
        WHERE model_name IS NOT NULL
        GROUP by model_name, brand_name, charge_power, battery_capacity, range, vehicle_category 
        ORDER BY vehicle_category"""

# get processed data
data = pd.read_sql(sql, engine)

# save table in database
data.to_sql("vehicles_types", engine, schema="msc_2023_dominik", if_exists='fail')


### Vehicle Information table: ###
# create final vehicle information table; containing all vehicle informatino for each car in the discrete tables
sql = """
WITH vehicles AS (
    SELECT vehicle_no, vehicle_category
    FROM msc_2023_dominik.vehicles_preprocessing 
), 
vehicle_categories AS (
    SELECT *
    FROM msc_2023_dominik.vehicles_types
)
SELECT *
FROM vehicles JOIN vehicle_categories USING(vehicle_category)
"""

# get processed data
data = pd.read_sql(sql, engine)

# remove index feature
data = data.drop(["index"], axis = 1)

# save table in database
data.to_sql("vehicle_information", engine, schema="msc_2023_dominik", if_exists='fail')

420

# Precalculation of Next Planned Booking and Next Planned Booking Duration Tables

This calculation can take more than 24h!

In [ ]:
# calculation is based on special version of car-sharing simulation environment
class CarsharingEnv:
    def __init__(self, stations, vehicle_information, episode_len = 24, dt=0.25, 
                 cancellation_penalty = 100, penalty_per_kwh = 0.25, v2g = True, 
                 v2g_demand_event = 500, v2g_max_duration = 3.0, v2g_penalty = 10000, 
                 v2g_probability_charging_event = 0.5, v2g_probability_discharging_event = 0.5, 
                 v2g_morning_time_period = [6.0, 9.0, 10.75], v2g_noon_time_period = [11.0, 14.0, 16.0], 
                 v2g_evening_time_period = [16.25, 19.0, 24.0], v2g_reward = 2000,
                 planned_bookings = True, max_distance_car_assingment = 5000, plot_state = True):
        """
        Parameters
        ----------
        stations: Geopandas Geodataframe
            Locations of car-sharing stations, including a distinct "station_no" attribute with station ID.
        vehicle_information: Pandas Dataframe
            Includes the features "vehicle_no", "charge_power", "battery_capacity", and "vehicle_category" for each car.
        episode_len: int, optional
            Length of one episode in hours, by default 24.
        dt: float, optional
            Time step size in hours, by default 0.25 (a quarter hour).
        cancellation_penalty: int, optional
            Maximum penalty in CHF for a booking cancelled due to not enough charged battery, by default 100.
        penalty_per_kwh: int, optional
            Penalty in CHF/kWh for total negative energy difference between beginning and ending of episode, by default 0.25.
        v2g: boolean, optional
            Boolean indicating if V2G events take place, by default True.
        v2g_demand_event: int, optional
            Energy demand during V2G event in kWh per time step (dt), by default 500.
        v2g_max_duration: int, optional
            Maximum duration of V2G charging or discharging event in hours, by default 3.
        V2G_penalty: int, optional
            Penalty in CHF if agent charges/discharges less energy than specified in "v2g_demand_event" during V2G event, by default 10000.
        v2g_probability_charging_event: float, optional
            Probability that a charging event will take place around noon, by default 0.5.
        v2g_probability_discharging_event: float, optional
            Probability that a discharging event will take place in the morning or evening, by default 0.5.
        v2g_morning_time_period: list, optional
            List containing: 1) first possible time (hours) for starting v2g discharging event in the morning, by default 6.0 (6 AM).
                             2) last possible time (hours) for starting v2g discharging event in the morning, by default 9.0 (9 AM).
                             3) last possible timestamp for v2g discharging operation, by default 10.75 (10:45 AM).
        v2g_noon_time_period: list, optional
            List containing: 1) first possible time (hours) for starting v2g charging event at noon, by default 11.0 (11 AM).
                             2) last possible time (hours) for starting v2g charging event at noon, by default 14.0 (2 PM).
                             3) last possible timestamp for v2g charging operation, by default 16.0 (4 PM).
        v2g_evening_time_period: list, optional
            List containing: 1) first possible time (hours) for starting v2g discharging event in the evening, by default 16.25 (4:15 PM).
                             2) last possible time (hours) for starting v2g discharging event in the evening, by default 19.0 (7 PM).
                             3) last possible timestamp for v2g charging operation, by default 24 (00:00 AM, next day).
        v2g_reward: int, optional
            Revenue during v2g operations per timestep in CHF, by default 2000 CHF.
        planned_bookings: boolean, optional
            Boolean indicating whether there are planned bookings in the environment; otherwise, all bookings are spontaneous, by default True.
        max_distance_car_assingment: int, optional
            Maximum search distance in meter for Car-assingment problem, by default 5000.
        plot_state: boolean, optional
            Plot current state of enviorment or not, by defualt True.
        ----------
        Observation space: 5 parts:
        1) Location for each car (three options):
            - Station number (1000-5000)
            - Reservation number (2x'xxx'xxx) during trip
            - Reservation number (3x'xxx'xxx) during relocation
            - -1 if car is not available
        2) State of charge (SOC) for each vehicle (between 0 and 1).
        3) Timestamp of the next planned booking for each car (discrete between 0 and self.episode_len).
        4) Duration of the next planned booking (measured in the number of time steps of length self.dt).
        5) Binary variable indicating the occurrence of a vehicle-to-grid (V2G) event.
        Remark: States 3) and 4) are only included if the variable "planned_bookings" is True.
        ----------
        Action space: 3 actions for each car:
        1) 0 = do nothing.
        2) 1 = charging.
        3) 3 = discharging (V2G).
        """

        # environment settings
        self.dt = dt
        self.episode_len = int(episode_len / self.dt) 
        self.planned_bookings = planned_bookings
        self.max_distance_car_assingment = max_distance_car_assingment
        self.plot_state = plot_state
        
        # stations in system
        self.stations = stations
        
        # vehicle information
        self.vehicles_id = vehicle_information["vehicle_no"]
        self.nr_vehicles = len(self.vehicles_id)
        self.chariging_power = vehicle_information["charge_power"]
        self.battery_capacities = vehicle_information["battery_capacity"] 
        self.vehicle_type = vehicle_information["vehicle_category"] 
        
        # define state boundaries for slicing list
        # locations upper bound
        self.locations_upper = self.nr_vehicles
        # soc upper bound
        self.soc_upper = 2*self.nr_vehicles
        # locations, upper bound:
        self.reservation_time_upper = self.nr_vehicles * 3
        self.v2g_lower = self.nr_vehicles * 4
        
        # Save discrete planned reservations
        self.planned_reservations_df_res = pd.DataFrame()
        self.planned_reservations_df_dur = pd.DataFrame()
            
        
    def reset(self, daily_data, reservations, week_nr, day, timesteps_since_start):
        """
        Parameters
        ----------
        daily_data: Pandas DataFrame
            Contains the car trips over the day.
        reservations: Pandas DataFrame
            Contains information about the cars, including "vehicle_no", "charge_power", "battery_capacity", and "vehicle_category".

        Returns
        ----------
        self.state: numpy ndarray
            The reset state of the environment at the first time step. It includes the following information for each car:
            1) Location:
                - Station number (1000-5000)
                - Reservation number (2x'xxx'xxx) during trip
                - Reservation number (3x'xxx'xxx) during relocation
                - -1 if the car is not available
            2) State of charge (SOC) for each vehicle (between 0 and 1).
            3) Timestamp of the next planned booking for each car (discrete value between 0 and self.episode_len).
            4) Duration of the next planned booking (measured in the number of time steps of length self.dt).
            5) Binary variable indicating the occurrence of a vehicle-to-grid (V2G) event.
            Note: The information in 3) and 4) is included only if the variable "planned_bookings" is True.
        """
        
        # set time to 0
        self.t = 0
        
        if week_nr == 0 and day == 98:
            self.week_nr = week_nr
            self.planned_reservations_df_res = pd.DataFrame()
            self.planned_reservations_df_dur = pd.DataFrame()
            self.planned_reservations_df_res["vehicle_no"] = self.vehicles_id
            self.planned_reservations_df_dur["vehicle_no"] = self.vehicles_id
        
        
        if week_nr != self.week_nr:
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
                self.planned_reservations_df_res.to_sql("planned_reservations_discrete_{}".format(week_nr-1), engine, schema="msc_2023_dominik", if_exists='replace')
                self.planned_reservations_df_dur.to_sql("planned_durations_discrete_{}".format(week_nr-1), engine, schema="msc_2023_dominik", if_exists='replace')
                with open('planned_reservations_week_{}.pickle'.format(self.week_nr), 'wb') as f:
                    # dump the dictionary to the file using pickle
                    pickle.dump(self.planned_reservations_car, f)
                self.planned_reservations_df_res = pd.DataFrame()
                self.planned_reservations_df_res["vehicle_no"] = self.vehicles_id
                self.planned_reservations_df_dur = pd.DataFrame()
                self.planned_reservations_df_dur["vehicle_no"] = self.vehicles_id
        # set week number
        self.week_nr = week_nr
        # reset reward lists 
        self.reward_list_trips = []
        self.reward_list_charging = []
        self.reward_list_cancellation_penalty = []
        self.reward_list_v2g = []
        self.reward_list = []
        
        # reset state
        self.state_old = np.zeros(self.nr_vehicles * 4 + 1)
        
        # reset planned reservations:
        self.planned_reservations_car = {key: [] for key in self.vehicles_id}
        
        
        
        ### initialize state at t = 0 ###
        
        # 1) car locations (three options)
        car_locations = daily_data.iloc[:,0].values
        
        # 2) SOC (state of charge) ##
        #car_SOC = np.random.rand(self.nr_vehicles) # random values between 0-100%
        #car_SOC = np.zeros(self.nr_vehicles) # all car batteries empty (0%)
        #car_SOC = np.full((self.nr_vehicles,), 0.2) # all car batteries at 20%
        #car_SOC = np.random.uniform(low=0.5, high=1, size=self.nr_vehicles) # All cars randomly between 50-100%
        np.random.seed(42)
        rng = np.random.RandomState(42)
        car_SOC = rng.uniform(low=0.5, high=1, size=self.nr_vehicles)
        
         # state 5) Binary V2G event
        v2g_event = np.array([0])
 
        # final state with planned bookings
        next_reservation, duration_next_reservation = self.update_reservation_state(reservations, True, timesteps_since_start)
            
        # concatinate states 1-5
        self.state = np.concatenate([car_locations, car_SOC, next_reservation, duration_next_reservation, v2g_event])
        
        return self.state
    
    
    def save_week(self, daily_data, reservations, week_nr, day, timesteps_since_start): 
        
        # if first week of simulation
        if week_nr == 0 and day == 98:
            self.week_nr = week_nr
            self.planned_reservations_df_res = pd.DataFrame()
            self.planned_reservations_df_dur = pd.DataFrame()
            self.planned_reservations_df_res["vehicle_no"] = self.vehicles_id
            self.planned_reservations_df_dur["vehicle_no"] = self.vehicles_id
        
        # if new week has started
        if week_nr != self.week_nr:
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
                self.planned_reservations_df_res.to_sql("planned_reservations_discrete_weeks_{}".format(week_nr-1), engine, schema="msc_2023_dominik", if_exists='fail')
                self.planned_reservations_df_dur.to_sql("planned_durations_discrete_weeks_{}".format(week_nr-1), engine, schema="msc_2023_dominik", if_exists='fail')
                with open('planned_reservations_week_{}.pickle'.format(self.week_nr), 'wb') as f:
                    # dump the dictionary to the file using pickle
                    pickle.dump(self.planned_reservations_car, f)
                self.planned_reservations_df_res = pd.DataFrame()
                self.planned_reservations_df_res["vehicle_no"] = self.vehicles_id
                self.planned_reservations_df_dur = pd.DataFrame()
                self.planned_reservations_df_dur["vehicle_no"] = self.vehicles_id

        # set week number
        self.week_nr = week_nr
        
        ### initialize state at t = 0 ###
        
        # 1) car locations (three options)
        #car_locations = daily_data.iloc[:,0].values
        
        # 2) SOC (state of charge) ##
        #car_SOC = np.random.rand(self.nr_vehicles) # random values between 0-100%
        #car_SOC = np.zeros(self.nr_vehicles) # all car batteries empty (0%)
        #car_SOC = np.full((self.nr_vehicles,), 0.2) # all car batteries at 20%
        #car_SOC = np.random.uniform(low=0.5, high=1, size=self.nr_vehicles) # All cars randomly between 50-100%
        #np.random.seed(42)
        #rng = np.random.RandomState(42)
        #car_SOC = rng.uniform(low=0.5, high=1, size=self.nr_vehicles)
        
         # state 5) Binary V2G event
        #v2g_event = np.array([0])
 
        # final state with planned bookings
        
        #next_reservation, duration_next_reservation = self.update_reservation_state(reservations, True)
            
            # concatinate states 1-5
        #self.state = np.concatenate([car_locations, car_SOC, next_reservation, duration_next_reservation, v2g_event])
        
        next_reservation, duration_next_reservation = self.update_reservation_state(reservations, False, timesteps_since_start)
        
        return 


    def update_reservation_state(self, reservations, reset, timesteps_since_start):
        """
         Parameters
        ----------
        reservations: Pandas DataFrame
            Includes the features "syscreatedate_daytime", "vehicle_no", "reservationfrom_daytime", and "reservation_duration" for each reservation.
        reset: boolean, optional
            Boolean indicating whether to reset the environment or not.

        Returns
        ----------
        next_reservation : numpy ndarray
            Timestamp of the next planned booking for each car (discrete value between 0 and self.episode_len).
        duration_next_reservation : numpy ndarray
            Duration of the next planned booking (measured in the number of time steps of length self.dt).
        """

        # initalize variables if during reset of environment
        if reset is True:
            time_ = 0
            next_reservation = np.ones(self.nr_vehicles) * -1
            duration_next_reservation = np.ones(self.nr_vehicles) * -1

        # get current state of next planned reservations (timestamp and duration)
        else:
            time_ = self.t + 1
            next_reservation = self.state[self.soc_upper : self.reservation_time_upper]
            duration_next_reservation = self.state[self.reservation_time_upper :self.v2g_lower]

        # filter reservations: 
        # bookings before t = 0 have value syscreatedate_daytime = 0
        reservations_t =  reservations[reservations["syscreatedate_time_discrete"] == time_]
        reservations_t.set_index(['vehicle_no'], inplace=True, drop=False)

        # iterate over all vehicles, search planned reservations
        count = 0

        for vehicle_id in self.vehicles_id:  
            
            # get reservation of car
            current_car = reservations_t[reservations_t["vehicle_no"] == vehicle_id]
            # remove reservations in past
            finish = False
            while finish is False:
                if self.planned_reservations_car[vehicle_id] and int(self.planned_reservations_car[vehicle_id][0][0]) <= self.t:
                    self.planned_reservations_car[vehicle_id] = self.planned_reservations_car[vehicle_id][1:]
                else:
                    finish = True
            
            # skip if no new reservation found
            if current_car.empty:
                
                # if current state's reservation is in the past 
                
                if next_reservation[count] <=  self.t and next_reservation[count] != -1:
                    
                    # check for planned reservations saved in the past
                    if self.planned_reservations_car[vehicle_id]:
                        
                        # save reservation timestamp
                        next_reservation[count] = self.planned_reservations_car[vehicle_id][0][0]
                        
                        # save reservation duration
                        duration_next_reservation[count] = self.planned_reservations_car[vehicle_id][0][1]
                        
                        # remove reservation from dict with planned reservations
                        #self.planned_reservations_car[vehicle_id] = self.planned_reservations_car[vehicle_id][1:]
                        
                    else:
                        # assign -1 for no planned reservation
                        next_reservation[count] = -1
                        duration_next_reservation[count] = -1
                        
                count +=1
                continue
                
            for i in range(0,len(current_car)):    

                # get timestamp and duration of next planned reservation
                reservation_time = current_car["reservationfrom_time_discrete"].iloc[i]
                reservation_duration = current_car["reservation_duration"].iloc[i]

                # save directly if during reset of environment
                if i == 0 and reset is True:
                    # save timestamp of next reservation
                    next_reservation[count] = reservation_time

                    # save reservation duration
                    duration_next_reservation[count] = reservation_duration


                if reset is True:
                    # save directly first reseration during reset of environment
                    if i == 0 and reset is True:
                        # save timestamp of next reservation
                        next_reservation[count] = reservation_time

                        # save reservation duration
                        duration_next_reservation[count] = reservation_duration
                        
                        new_reservation = [reservation_time, reservation_duration]

                        # remain order in dict (nearest reservation in first index)
                        index = bisect.bisect_left([sublist[0] for sublist in  self.planned_reservations_car[vehicle_id]], new_reservation[0])

                        # save timestamp and duration of new but later reservations in dict
                        self.planned_reservations_car[vehicle_id].insert(index, new_reservation)


                    # save other found reservation during reset of environment in dict
                    else:
                        new_reservation = [reservation_time, reservation_duration]

                        # remain order in dict (nearest reservation in first index)
                        index = bisect.bisect_left([sublist[0] for sublist in  self.planned_reservations_car[vehicle_id]], new_reservation[0])

                        # save timestamp and duration of new but later reservations in dict
                        self.planned_reservations_car[vehicle_id].insert(index, new_reservation)

                else: 
                    # save found reservation environment in dict
                    new_reservation = [reservation_time, reservation_duration]

                    # remain order in dict (nearest reservation in first index)
                    index = bisect.bisect_left([sublist[0] for sublist in  self.planned_reservations_car[vehicle_id]], new_reservation[0])

                    # save timestamp and duration of new but later reservations in dict
                    self.planned_reservations_car[vehicle_id].insert(index, new_reservation)


            # update only if reservation is before last planned reservation
            if  reset is False and self.planned_reservations_car[vehicle_id][0]:
                # save timestamp of next reservation
                next_reservation[count] = self.planned_reservations_car[vehicle_id][0][0]

                # save reservation duration
                duration_next_reservation[count] = self.planned_reservations_car[vehicle_id][0][1]

                # remove reservation from dict
                #self.planned_reservations_car[vehicle_id] = self.planned_reservations_car[vehicle_id][1:]

            count += 1
        
        self.planned_reservations_df_res[daily_data.columns[time_ - timesteps_since_start]] = next_reservation
        self.planned_reservations_df_dur[daily_data.columns[time_- timesteps_since_start]] = duration_next_reservation

        return next_reservation, duration_next_reservation


    def step(self, daily_data, reservations, timesteps_since_start):
        
        # special step function for calculating next planned bookings tables
        next_reservation, duration_next_reservation = self.update_reservation_state(reservations, False, timesteps_since_start)
        self.state[self.soc_upper : self.reservation_time_upper] = next_reservation
        self.state[self.reservation_time_upper :self.v2g_lower] = duration_next_reservation

        done = True if (self.t - timesteps_since_start) == (self.episode_len - 2) else False
        # update time step
        self.t += 1

        return self.state, done, {}



In [3]:
# get station geodata, create spatial index
sql = " SELECT * FROM msc_2023_dominik.distinct_stations"
stations = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
stations.sindex

# get vehicle data
sql = "SELECT * FROM msc_2023_dominik.vehicle_information ORDER BY vehicle_no"
vehicles = pd.read_sql(sql, engine)

# get reservation data
sql = "SELECT vehicle_no, reservationfrom_time_discrete, syscreatedate_time_discrete, reservation_duration FROM msc_2023_dominik.reservations_long_time ORDER BY reservationfrom_time_discrete"
reservations = pd.read_sql(sql, engine)
reservations.set_index(['vehicle_no', 'reservationfrom_time_discrete', 'syscreatedate_time_discrete'], inplace=True, drop=False)

In [4]:
# simulate full observation period for the precalculation of the next planned bookings tables

env = CarsharingEnv(stations, vehicles, planned_bookings = True, plot_state = False)

# number of days to simulate 
nr_iterations = 577

# number of vehicles
nr_vehicles = len(vehicles)

# maximal simulation length
if nr_iterations > 577:
    nr_iterations = 577
    
count = 0

# iterate over weeks (for loading weekly discrete data)
for week_nr in range(0, math.ceil(nr_iterations / 7)):
    # load discrete car-sharing table
    sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(week_nr)
    data = pd.read_sql(sql, engine)

    # iteration for each day
    for day in range(98,676,96):
        timesteps_since_start = count * 96
        
        # all requested days are simulated
        if count == nr_iterations:
            s = env.reset(daily_data, reservations, 83, day)
            break
            
        # get date
        date = pd.to_datetime(data.columns[day-97])
       
        # get discete data of day
        daily_data = data.iloc[:,day-97:day]   
        
        if week_nr == 0 and day == 98:
            s = env.reset(daily_data, reservations, week_nr, day, timesteps_since_start)
        else:
            # reset day at begining of new episode (day)
            env.save_week(daily_data, reservations, week_nr, day, timesteps_since_start)
            
        # simulate day in 15 min steps
        done = False
        counter = 0
        while not done:
            # proceed one time step
            s, done, _ = env.step(daily_data, reservations, timesteps_since_start)
        
            counter +=1
            
        #with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
            #print(env.planned_reservations_df_res.head(100))
            
        print("Week: ", week_nr," Day: ", day)

        count += 1

Week:  0  Day:  98
Week:  0  Day:  194
Week:  0  Day:  290
Week:  0  Day:  386
Week:  0  Day:  482
Week:  0  Day:  578
Week:  0  Day:  674
Week:  1  Day:  98
Week:  1  Day:  194
Week:  1  Day:  290
Week:  1  Day:  386
Week:  1  Day:  482
Week:  1  Day:  578
Week:  1  Day:  674
Week:  2  Day:  98
Week:  2  Day:  194
Week:  2  Day:  290
Week:  2  Day:  386
Week:  2  Day:  482
Week:  2  Day:  578
Week:  2  Day:  674
Week:  3  Day:  98
Week:  3  Day:  194
Week:  3  Day:  290
Week:  3  Day:  386
Week:  3  Day:  482
Week:  3  Day:  578
Week:  3  Day:  674
Week:  4  Day:  98
Week:  4  Day:  194
Week:  4  Day:  290
Week:  4  Day:  386
Week:  4  Day:  482
Week:  4  Day:  578
Week:  4  Day:  674
Week:  5  Day:  98
Week:  5  Day:  194
Week:  5  Day:  290
Week:  5  Day:  386
Week:  5  Day:  482
Week:  5  Day:  578
Week:  5  Day:  674
Week:  6  Day:  98
Week:  6  Day:  194
Week:  6  Day:  290
Week:  6  Day:  386
Week:  6  Day:  482
Week:  6  Day:  578
Week:  6  Day:  674
Week:  7  Day:  98
Week:  7

TypeError: reset() missing 1 required positional argument: 'timesteps_since_start'

# Preprocessing of Grid Balancing Energy Prices (V2G/G2V) and Energy Prices for Charging

In [10]:
### Grid Balancing Energy Prices

# load grid balancing energy prices of 2019
v2g_prices_2019_df = pd.read_csv("Data/EnergieUebersichtCH_2019.csv", sep=";", encoding='ISO-8859-1', header=0, skiprows=[1])

# rename features
v2g_prices_2019_df.rename(columns={'Unnamed: 0': 'Timestamp', 
                                   'Durchschnittliche negative Sekundär-Regelenergie Preise\nAverage negative secondary control energy prices': 'Secondary_negative_v2g_prices_chf_kwh',
                                   'Durchschnittliche positive Sekundär-Regelenergie Preise\nAverage positive secondary control energy prices': 'Secondary_positive_v2g_prices_chf_kwh'}, inplace = True)

# transform to datetime
v2g_prices_2019_df['Timestamp'] = pd.to_datetime(v2g_prices_2019_df['Timestamp'])

# exchange rates CHF - Euro
# Source: https://www.estv.admin.ch/estv/de/home/mehrwertsteuer/mwst-abrechnen/mwst-fremdwaehrungskurse/archiv-der-monatsmittelkurse/archiv-2019.html, accessed on 14.06.2023
conversion_rates_2019 = {
    1: 1.1414,
    2: 1.1393,
    3: 1.1478,
    4: 1.1463,
    5: 1.1387,
    6: 1.1461,
    7: 1.1302,
    8: 1.1208,
    9: 1.1037,
    10: 1.1021,
    11: 1.1051,
    12: 1.1095
}

# transform prices from Euro/MWh to CHF/kWh
for month, conversion_rate in conversion_rates_2019.items():
    rows = (v2g_prices_2019_df['Timestamp'].dt.month == month)
    v2g_prices_2019_df.loc[rows, 'Secondary_negative_v2g_prices_chf_kwh'] *= conversion_rate / 1000
    v2g_prices_2019_df.loc[rows, 'Secondary_positive_v2g_prices_chf_kwh'] *= conversion_rate / 1000

    
# load Grid Balancing Energy Prices of 2020 
v2g_prices_2020_df = pd.read_csv("Data/EnergieUebersichtCH_2020.csv", sep=";", encoding='ISO-8859-1', header=0, skiprows=[1])
v2g_prices_2020_df.rename(columns={'Unnamed: 0': 'Timestamp', 
                                   'Durchschnittliche negative Sekundär-Regelenergie Preise\nAverage negative secondary control energy prices': 'Secondary_negative_v2g_prices_chf_kwh',
                                   'Durchschnittliche positive Sekundär-Regelenergie Preise\nAverage positive secondary control energy prices': 'Secondary_positive_v2g_prices_chf_kwh'}, inplace = True)

# transform to datetime
v2g_prices_2020_df['Timestamp'] = pd.to_datetime(v2g_prices_2020_df['Timestamp'])

# exchange rates CHF - Euro
# Source: https://www.estv.admin.ch/estv/de/home/mehrwertsteuer/mwst-abrechnen/mwst-fremdwaehrungskurse/archiv-der-monatsmittelkurse/archiv-2020.html, accessed on 14.06.2023
conversion_rates_2020 = {
    1: 1.1062,
    2: 1.0912,
    3: 1.0776,
    4: 1.0712,
    5: 1.0667,
    6: 1.0658,
    7: 1.0818,
    8: 1.0795,
    9: 1.0886,
    10: 1.0887,
    11: 1.0870,
    12: 1.0859
}

# transform prices from Euro/MWh to CHF/kWh
for month, conversion_rate in conversion_rates_2020.items():
    rows = (v2g_prices_2020_df['Timestamp'].dt.month == month)
    v2g_prices_2020_df.loc[rows, 'Secondary_negative_v2g_prices_chf_kwh'] *= conversion_rate / 1000
    v2g_prices_2020_df.loc[rows, 'Secondary_positive_v2g_prices_chf_kwh'] *= conversion_rate / 1000

# merge both tables
v2g_prices = pd.concat([v2g_prices_2019_df, v2g_prices_2020_df], axis=0)

# save table in database 
v2g_prices.to_sql("v2g_prices", engine, schema="msc_2023_dominik", if_exists='fail')



### Energy Prices for Charging
# dataset: https://www.epexspot.com/en/market-data?market_area=CH&trading_date=2023-04-27&delivery_date=2023-04-28&underlying_year=&modality=Auction&sub_modality=DayAhead&technology=&product=60&data_mode=table&period=&production_period=

# load data 2019
energy_prices_2019_df = pd.read_csv("Data/auction_spot_prices_switzerland_2019.csv", sep=",", encoding='ISO-8859-1', header=1)

# transform feature to datetime
energy_prices_2019_df['Delivery day'] = pd.to_datetime(energy_prices_2019_df['Delivery day'], format='%d/%m/%Y')

# rename features
energy_prices_2019_df.rename(columns={'Hour 3A': 'Hour 3'}, inplace = True)

# fill Null values with prices one hour before 
energy_prices_2019_df['Hour 3'].fillna(energy_prices_2019_df['Hour 2'], inplace=True)


# load data 2020
energy_prices_2020_df = pd.read_csv("Data/auction_spot_prices_switzerland_2020.csv", sep=",", encoding='ISO-8859-1', header=1)

# transform feature to datetime
energy_prices_2020_df['Delivery day'] = pd.to_datetime(energy_prices_2020_df['Delivery day'], format='%d/%m/%Y')

# rename features
energy_prices_2020_df.rename(columns={'Hour 3A': 'Hour 3'}, inplace = True)

# fill Null values with prices one hour before
energy_prices_2020_df['Hour 3'].fillna(energy_prices_2020_df['Hour 2'], inplace=True)

# merge both tables
energy_prices = pd.concat([energy_prices_2019_df, energy_prices_2020_df], axis=0)
energy_prices = energy_prices[energy_prices['Delivery day'] >= pd.to_datetime("2019-01-01")]

# caculate prices in 0.25h steps per kwh
hours_list = energy_prices.columns.tolist()[1:26]
hours_list.remove('Hour 3B')
        
# Discretize prices, transform from Euro/MGW to CHF/kWh
for year in range(2019, 2021):
    if year == 2019:
        dict_rate = conversion_rates_2019
    if year == 2020:
        dict_rate = conversion_rates_2020
    for month in range(1, 13):
        selection = energy_prices[(energy_prices['Delivery day'].dt.year == year) & (energy_prices['Delivery day'].dt.month == month)]
        for hour in hours_list:
            price = energy_prices[hour] / 1000 * dict_rate[month]
            for i in range(0,4):
                energy_prices["Price_chf_kwh_{}".format(int(hour[5:])-1 + i/4)] = price 

# save table in database 
energy_prices.to_sql("charging_costs", engine, schema="msc_2023_dominik", if_exists='fail')


731

In [22]:
### Energy Prices for Charging (for Elie's thesis)
# dataset: https://www.epexspot.com/en/market-data?market_area=CH&trading_date=2023-04-27&delivery_date=2023-04-28&underlying_year=&modality=Auction&sub_modality=DayAhead&technology=&product=60&data_mode=table&period=&production_period=

# exchange rates CHF - Euro
# Source: https://www.estv.admin.ch/estv/de/home/mehrwertsteuer/mwst-abrechnen/mwst-fremdwaehrungskurse/archiv-der-monatsmittelkurse/archiv-2018.html, accessed on 14.06.2023
conversion_rates_2018 = {
    1: 1.1792,
    2: 1.1869,
    3: 1.1686,
    4: 1.1758,
    5: 1.1963,
    6: 1.2015,
    7: 1.1676,
    8: 1.1724,
    9: 1.1576,
    10: 1.1411,
    11: 1.1524,
    12: 1.1510
}

# load data 2018
energy_prices_2018_df = pd.read_csv("Data/auction_spot_prices_switzerland_2018.csv.gz", sep=",", encoding='ISO-8859-1', header=1)

# transform feature to datetime
energy_prices_2018_df['Delivery day'] = pd.to_datetime(energy_prices_2018_df['Delivery day'], format='%d/%m/%Y')

# rename features
energy_prices_2018_df.rename(columns={'Hour 3A': 'Hour 3'}, inplace = True)

# fill Null values with prices one hour before
energy_prices_2018_df['Hour 3'].fillna(energy_prices_2018_df['Hour 2'], inplace=True)
energy_prices = energy_prices_2018_df
energy_prices = energy_prices[energy_prices['Delivery day'] >= pd.to_datetime("2018-01-01")].copy()

# caculate prices in 0.25h steps per kwh
hours_list = energy_prices.columns.tolist()[1:26]
hours_list.remove('Hour 3B')
        
# Discretize prices, transform from Euro/MGW to CHF/kWh
for year in range(2018, 2019):
    if year == 2018:
        dict_rate = conversion_rates_2018
    for month in range(1, 13):
        selection = energy_prices[(energy_prices['Delivery day'].dt.year == year) & (energy_prices['Delivery day'].dt.month == month)]
        for hour in hours_list:
            price = energy_prices[hour] / 1000 * dict_rate[month]
            for i in range(0,4):
                energy_prices["Price_chf_kwh_{}".format(int(hour[5:])-1 + i/4)] = price 

# save table in database                 
energy_prices.to_sql("charging_costs_2018", engine, schema="msc_2023_dominik", if_exists='fail')


365